In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# setup PYTHONPATH
import sys
sys.path += ['.', './extern/scannetpp', './extern/mast3r', './extern/mast3r/dust3r']

In [3]:
from pathlib import Path
import torch
from torchvision.io import read_image

import pyvista as pv
pv.set_jupyter_backend('trame')

from dataset import SceneDataset, SceneDatasetTransformToTorch
from einops import rearrange
from models.surface_net_baseline.model import SimpleOccNetConfig
from models.surface_net_baseline.module import LRConfig, OccSurfaceNet, OptimizerConfig
from models.surface_net_baseline.util import project_points_to_images
from utils.data_parsing import load_yaml_munch
from lightning import Trainer
from lightning.pytorch.loggers import WandbLogger
import lightning as pl

from utils.visualize import visualize_mesh
from utils.basic import get_default_device


In [4]:
# Init dataset and load for visualization
dataset = SceneDataset(
    data_dir="datasets/scannetpp/data",
    camera="iphone",
    n_points=300000,
    threshold_occ=0.01,
    representation="occ",
    visualize=False,
)

device = get_default_device()
transform = SceneDatasetTransformToTorch(target_device=device)

In [5]:
idx = dataset.get_index_from_scene("8b2c0938d6")
data = dataset[idx]

image_names, camera_params_list, _ = data["images"]
images, transformations, points, gt = transform.forward(data)
# and normalize images
images = images / 255.0

/Users/luca/uni/master/dl-in-vc/voxl3r/dataset.py:301: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  points = torch.tensor(torch.from_numpy(points).float()).to(self.target_device)
/Users/luca/uni/master/dl-in-vc/voxl3r/dataset.py:302: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gt = torch.tensor(torch.from_numpy(gt).float()).to(self.target_device)


In [6]:
from dataset import plot_training_example

plot_training_example(dataset, idx)

Widget(value='<iframe src="http://localhost:56725/index.html?ui=P_0x28c0d7b00_0&reconnect=auto" class="pyvista…

In [7]:

from dataset import plot_mask

plot_mask(dataset, idx)

100%|██████████| 551/551 [01:07<00:00,  8.14it/s]


Widget(value='<iframe src="http://localhost:56725/index.html?ui=P_0x2de61fe90_1&reconnect=auto" class="pyvista…

In [8]:
from dataset import plot_occupency_grid

plot_occupency_grid(dataset, idx)

Voxel Grid Size (L, W, H): (np.float64(1.9772891040177676), np.float64(2.392200282379526), np.float64(2.1013366641606663))
Voxel Grid Center: [4.07553703 1.72495476 0.60615678]
Occupied Voxels: 12552.0


Widget(value='<iframe src="http://localhost:56725/index.html?ui=P_0x17fef84a0_2&reconnect=auto" class="pyvista…

2024-11-07 18:40:07.620 Python[47377:22128909] GLDRendererMetal command buffer completion error: Error Domain=MTLCommandBufferErrorDomain Code=1 "Discarded (victim of GPU error/recovery) (00000005:kIOGPUCommandBufferCallbackErrorInnocentVictim)" UserInfo={NSLocalizedDescription=Discarded (victim of GPU error/recovery) (00000005:kIOGPUCommandBufferCallbackErrorInnocentVictim), NSUnderlyingError=0x10878f7f0 {Error Domain=IOGPUCommandQueueErrorDomain Code=5 "(null)"}}


: 